In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder,OrdinalEncoder,StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import warnings
from sklearn.pipeline import Pipeline
import pickle


In [7]:
warnings.filterwarnings('ignore')
test=pd.read_csv("test.csv")
test.head()


,customer_id,Name,age,gender,security_no,region_category,membership_category,joining_date,joined_through_referral,referral_id,...,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,fffe43004900440033003200320035003600,Tobias Liebold,24,F,I4AYTC2,City,Premium Membership,2015-04-22,No,xxxxxxxx,...,17,101.50,32593.20,15.0,801.18,Yes,No,No,Not Applicable,Products always in Stock
1,fffe43004900440032003200350035003400,Patrick Kizer,53,F,WV0LB6W,Town,Silver Membership,2016-01-19,No,xxxxxxxx,...,12,324.61,39155.49,21.0,NaN,No,Yes,No,Not Applicable,No reason specified
2,fffe43004900440031003000380038003300,Annamaria Freese,53,F,94O1F22,Town,No Membership,2016-02-07,Yes,CID19334,...,20,47.71,35434.17,12.0,675.17,Yes,No,No,Not Applicable,Poor Product Quality
3,fffe43004900440034003200300031003800,Gilda Lundy,61,M,74WFG9K,NaN,Gold Membership,2017-10-24,No,xxxxxxxx,...,19,451.66,30621.93,7.0,755.93,Yes,Yes,Yes,Solved,Poor Product Quality
4,fffe43004900440034003100380030003300,Angla Alameda,46,F,249HVEX,Town,Premium Membership,2016-06-11,No,xxxxxxxx,...,12,266.68,50462.15,Error,806.67,Yes,Yes,Yes,Solved,Products always in Stock


In [36]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7399 entries, 0 to 7398
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   customer_id                   7399 non-null   object 
 1   Name                          7399 non-null   object 
 2   age                           7399 non-null   int64  
 3   gender                        7399 non-null   object 
 4   security_no                   7399 non-null   object 
 5   region_category               6337 non-null   object 
 6   membership_category           7399 non-null   object 
 7   joining_date                  7399 non-null   object 
 8   joined_through_referral       7399 non-null   object 
 9   referral_id                   7399 non-null   object 
 10  preferred_offer_types         7339 non-null   object 
 11  medium_of_operation           7399 non-null   object 
 12  internet_option               7399 non-null   object 
 13  las

In [37]:
test.drop(["customer_id","Name","security_no","referral_id","joining_date","last_visit_time",'internet_option'],axis="columns",inplace=True)

In [38]:
test.head()

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,24,F,City,Premium Membership,No,Gift Vouchers/Coupons,Smartphone,17,101.50,32593.20,15.0,801.18,Yes,No,No,Not Applicable,Products always in Stock
1,53,F,Town,Silver Membership,No,Without Offers,Smartphone,12,324.61,39155.49,21.0,NaN,No,Yes,No,Not Applicable,No reason specified
2,53,F,Town,No Membership,Yes,Without Offers,Smartphone,20,47.71,35434.17,12.0,675.17,Yes,No,No,Not Applicable,Poor Product Quality
3,61,M,NaN,Gold Membership,No,Gift Vouchers/Coupons,Both,19,451.66,30621.93,7.0,755.93,Yes,Yes,Yes,Solved,Poor Product Quality
4,46,F,Town,Premium Membership,No,Credit/Debit Card Offers,Smartphone,12,266.68,50462.15,Error,806.67,Yes,Yes,Yes,Solved,Products always in Stock


In [39]:
test=test[test["joined_through_referral"].isin(["Yes","No"])]
test=test[test["gender"].isin(["M","F"])]
test=test[test["medium_of_operation"] != '?']
test["avg_frequency_login_days"]=test["avg_frequency_login_days"].apply(lambda x: float(x) if x!='Error' else None)
test["avg_frequency_login_days"].fillna(test["avg_frequency_login_days"].mode()[0],inplace=True)
test["region_category"].fillna(test["region_category"].mode()[0],inplace=True)
test["preferred_offer_types"].fillna(test["preferred_offer_types"].mode()[0],inplace=True)

In [40]:
test.head()

,age,gender,region_category,membership_category,joined_through_referral,preferred_offer_types,medium_of_operation,days_since_last_login,avg_time_spent,avg_transaction_value,avg_frequency_login_days,points_in_wallet,used_special_discount,offer_application_preference,past_complaint,complaint_status,feedback
0,24,F,City,Premium Membership,No,Gift Vouchers/Coupons,Smartphone,17,101.50,32593.20,15.0,801.18,Yes,No,No,Not Applicable,Products always in Stock
1,53,F,Town,Silver Membership,No,Without Offers,Smartphone,12,324.61,39155.49,21.0,NaN,No,Yes,No,Not Applicable,No reason specified
2,53,F,Town,No Membership,Yes,Without Offers,Smartphone,20,47.71,35434.17,12.0,675.17,Yes,No,No,Not Applicable,Poor Product Quality
3,61,M,Town,Gold Membership,No,Gift Vouchers/Coupons,Both,19,451.66,30621.93,7.0,755.93,Yes,Yes,Yes,Solved,Poor Product Quality
4,46,F,Town,Premium Membership,No,Credit/Debit Card Offers,Smartphone,12,266.68,50462.15,7.0,806.67,Yes,Yes,Yes,Solved,Products always in Stock


In [41]:

test["avg_frequency_login_days"]=test['avg_frequency_login_days'].apply(lambda x:abs(x))
test["avg_time_spent"]=test['avg_time_spent'].apply(lambda x:abs(x))
test["days_since_last_login"]=test["days_since_last_login"].apply(lambda x: abs(x) )

In [42]:
numeric_cols = test.select_dtypes(include=np.number).columns

for col in numeric_cols:
    Q1 = test[col].quantile(0.20)
    Q3 = test[col].quantile(0.80)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    test = test[(test[col] >= lower) & (test[col] <= upper)]

In [43]:
ct=ColumnTransformer(transformers=[
    ('tnf1',OneHotEncoder(sparse_output=False,drop='first'),['gender','region_category','joined_through_referral','preferred_offer_types','medium_of_operation']),

    ('tnf2',OrdinalEncoder(categories=[['No Membership','Basic Membership','Premium Membership','Silver Membership','Gold Membership','Platinum Membership']]),['membership_category']),

    ('tnf3',OrdinalEncoder(categories=[['Yes','No'],['Yes','No'],['Yes','No']]),['used_special_discount','offer_application_preference','past_complaint']),

    ('tnf4',OrdinalEncoder(categories=[['User Friendly Website','Products always in Stock','Reasonable Price','Quality Customer Care','Poor Website','Too many ads','Poor Customer Service','No reason specified','Poor Product Quality']]),['feedback']),

    ('tnf5',OrdinalEncoder(categories=[['Unsolved','Not Applicable', 'No Information Available','Solved in Follow-up', 'Solved']]),['complaint_status']),
    ('tnf6',StandardScaler(), ['age','avg_transaction_value', 'avg_time_spent', 'points_in_wallet', 'days_since_last_login', 'avg_frequency_login_days'])

    ],remainder='passthrough'

)

In [44]:
x=ct.fit_transform(test)

In [45]:
x[:5]

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  2.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         1.00000000e+00,  1.00000000e+00, -8.49158003e-01,
         2.41594209e-01, -6.33262668e-01,  1.06089895e+00,
         7.40339121e-01, -3.55867742e-02],
       [ 0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         1.00000000e+00,  0.00000000e+00,  1.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  1.00000000e+00,
         8.00000000e+00,  1.00000000e+00,  9.93851240e-01,
         3.97558447e-01, -8.39245665e-01, -6.41025758e-02,
         1.26864455e+00, -3.96611110e-01],
       [ 1.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  4.00000000e+00,
         0.00000000e+00,  0.0

In [46]:
with open('model_pkl','rb') as f:
    loaded_model=pickle.load(f)

In [47]:
predictions=loaded_model.predict(x)

In [48]:
predictions[:10]

array([1, 5, 3, 1, 5, 3, 2, 1, 5, 3])

In [49]:
len(predictions)

3960